In [1]:
import os
import pyreadstat
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use dask/ray
# import modin.pandas as pd
import pandas as pd
from datetime import datetime
from collections import Counter
import pickle
from dask.distributed import Client
from dask.diagnostics import ProgressBar
from functools import partial
from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor
from icecream import ic
#import memory_profiler
from memory_profiler import profile
%load_ext memory_profiler

#own imports
from dhs_pca_functions import *
from dhs_gpt import *

In [2]:
folder_path = "/mnt/datadisk/data/surveys/DHS_final_raw_data/"
work_dir = "/mnt/datadisk/data/Projects/water/pickles/"
input_dir = "/mnt/datadisk/data/Projects/water/inputs/"
min_version = 8
max_version = False
overwrite_pqt = False
parquet = False
use_dask = False
drop_threshold = False
drop_col_counter = False
cols_to_load_simultaneously = 15
dataset_type = 'BR'  #, 'BR', 'CR', 'HW', 'IR', 'KR', 'MR', 'PR', 'AR'] ??['AN', 'FC, 'FP', 'HW', 'PV', 'SC'] < 26 surveys
add_str = 'pd'
columns = []
verbose = False
available_since_v = 7
pqt_file = f"{work_dir}New_DHS_{dataset_type}_data_V{min_version}_cols_{drop_col_counter}_{add_str}_new"
if parquet:
    pqt_file += '.pqt'
else:
    pqt_file += '.pkl'
further_pqt_files = f"{work_dir}DHS_{dataset_type}_data_V{min_version}_cols_{drop_col_counter}_{add_str}_"
excl_countries = [] #['EG', 'ZA']

In [3]:
# Load column names from all .sav files
file_paths = get_dhs_files(folder_path, dataset_type, min_version, excl_countries=excl_countries, max_version=max_version)
file_paths = sorted(file_paths)
ic(len(file_paths))
ic(file_paths[0])

'/mnt/datadisk/data/surveys/DHS_final_raw_data/BFBR81SV/BFBR81FL.SAV'

In [4]:
further_pqt_file = further_pqt_files + 'cols.pkl'
if not os.path.exists(further_pqt_file) or overwrite_pqt:
    common_column_labels, common_column_names, missing_label_details, uncommon_column_names, uncommon_column_labels, all_col_names, all_col_labels = get_common_columns_new(file_paths)
    with open(further_pqt_file, 'wb') as f:
        pickle.dump([common_column_labels, common_column_names, missing_label_details, uncommon_column_names, uncommon_column_labels, all_col_names, all_col_labels], f)
else:
    with open(further_pqt_file, 'rb') as f:
        common_column_labels, common_column_names, missing_label_details, uncommon_column_names, uncommon_column_labels, all_col_names, all_col_labels = pickle.load(f)

In [5]:
summary_f = input_dir + 'summary_df_V' + str(min_version) + f'_{dataset_type}_' + '.pkl'
#overwrite_pqt = True
if not os.path.exists(summary_f) or overwrite_pqt:
    summary_df = create_summary_df(file_paths, dataset_type, all_col_labels, cols_to_load_simultaneously, folder_path, create_GEID=False, create_DHSID=False, available_since_v=available_since_v)
    summary_df.to_csv(summary_f[:-4] + '.csv', index=False)
    summary_df.to_pickle(summary_f)
else:
    summary_df = pd.read_pickle(summary_f)
    #summary_df = summary_df.sort_index()
summary_df


  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:04<?, ?it/s]


ValueError: non unique cluster + hh nr and non unique case identification - skipping survey

In [ ]:
# overwrite_pqt = True
summary_f_gpt = summary_f[:-4] + '_unify_answers.pkl'
if not os.path.exists(summary_f_gpt) or overwrite_pqt:
    summary_df = unify_answers_iterator_wrapper(summary_df, input_dir)
    # ic(summary_df.shape)
    # ic(summary_df['column_name'])
    sorted_cols = ['similar_columns nr', 'column_name', 'column_name_normalized', 'numeric values as keys decision', 
                'available_data [%]', 'numeric [%]', 'string [%]', 'surveys with data in %', 'surveys with data percentage data available','available_data V7+ [%]', 'unique_answers wo keys',
                'unified answers 0', 'replace_d unified answers 0', 'unified answers not found 0', 'unified answers not clustered 0',
                'unified answers 1', 'replace_d unified answers 1', 'unified answers not found 1', 'unified answers not clustered 1',
                'unified answers 2', 'replace_d unified answers 2', 'unified answers not found 2', 'unified answers not clustered 2',
                'unified answers 3', 'replace_d unified answers 3', 'unified answers not found 3', 'unified answers not clustered 3',
                'unified answers 4', 'replace_d unified answers 4', 'unified answers not found 4', 'unified answers not clustered 4',
                'unified answers 5', 'replace_d unified answers 5', 'unified answers not found 5', 'unified answers not clustered 5',
                'unified answers 6', 'replace_d unified answers 6', 'unified answers not found 6', 'unified answers not clustered 6',
                'unified answers 7', 'replace_d unified answers 7', 'unified answers not found 7', 'unified answers not clustered 7',
                'original % overwriting (as % of col2)', 
                'original % different values (as % of col2) wo keys', 
                'cosine string similarity answers', 'original different value counts', 
                'answer mapping for integration', 'answer mapping for integration not found',  'unique_answers',
                'integrated % overwriting (as % of col2)', 'integrated % different values (as % of col2) wo keys', 
                'integrated different value counts', 'replace_d numeric values as keys', 'numeric values as keys not found', 'overall indicator',  
                'numeric similarity', 'cosine string similarity indicator', 'mean', 'std', 'string similarity indicator', 'string similarity of non matching answers',
                ] 
    for c in sorted_cols.copy():
        if c not in summary_df.columns:
            sorted_cols.remove(c)
    cols = sorted_cols + [c for c in summary_df.columns if c not in sorted_cols]
    summary_df = summary_df[cols]
    
    # sort by available data
    summary_df = summary_df.sort_values(by=['available_data [%]'], ascending=False)
    summary_df.to_csv(summary_f_gpt[:-4] + '.csv', index=False)
    summary_df.to_pickle(summary_f_gpt)

else:
    summary_df = pd.read_pickle(summary_f_gpt)

In [ ]:
# overwrite_pqt = True
integration_summary_f = input_dir + 'summary_df_V' + str(min_version) + \
    f'_{dataset_type}_clustered' + '.pkl'
#summary_f_clustering = input_dir + 'summary_df_V' + str(min_version) + \
 #   f'_{dataset_type}_unclustered' + '.pkl'
if not os.path.exists(integration_summary_f) or overwrite_pqt:
    integration_summary_df = summary_df[summary_df['available_data [%]'] >= 0.2]
    ic(integration_summary_df.shape)
    integration_summary_df = retrieve_embeddings(integration_summary_df, input_dir)
    ic(integration_summary_df.shape)

    sorted_cols = ['similar_columns nr', 'column_name', 'column_name_normalized', 'numeric values as keys decision', 
                   'available_data [%]', 'numeric [%]', 'string [%]', 'surveys with data in %', 'surveys with data percentage data available','available_data V7+ [%]',
                'original % overwriting (as % of col2)', 
                'original % different values (as % of col2) wo keys', 
                'cosine string similarity answers', 'original different value counts', 
                'answer mapping for integration', 'answer mapping for integration not found', 'unique_answers wo keys',
                'integrated % overwriting (as % of col2)', 'integrated % different values (as % of col2) wo keys', 
                'integrated different value counts', 'replace_d numeric values as keys', 'numeric values as keys not found', 'overall indicator',  
                'numeric similarity', 'cosine string similarity indicator', 'mean', 'std', 'string similarity indicator', 'string similarity of non matching answers',
                ] 
    for c in sorted_cols.copy():
        if c not in integration_summary_df.columns:
            sorted_cols.remove(c)
    ic(sorted_cols)
    cols = sorted_cols + [c for c in integration_summary_df.columns if c not in sorted_cols]
    integration_summary_df = integration_summary_df[cols]    
    integration_summary_df = integration_summary_df[integration_summary_df['similar_columns nr'] != -1]
    integration_summary_df = integration_summary_df.reset_index(drop=True)
    integration_summary_df_csv = integration_summary_df.drop(columns=['ada_embedding'])
    integration_summary_df_csv.to_csv(integration_summary_f[:-4] + '.csv', index=False)
    integration_summary_df.to_pickle(integration_summary_f)
else:
    integration_summary_df = pd.read_pickle(integration_summary_f)
integration_summary_df    

,similar_columns nr,column_name,column_name_normalized,numeric values as keys decision,available_data [%],numeric [%],string [%],surveys with data in %,surveys with data percentage data available,available_data V7+ [%],...,data_type,max,min,dubious_answers,unique_answers amount,unique_answers wo keys list,unique_answers wo keys dict,replace_d corrected_answers,replace_d corrected_answers w keys,ada_embedding
0,61,number of household members,number,NaN,100.000,100.0,0.0,100.00,100.0,100.000,...,numeric,98.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"[-0.007650680374354124, -0.01233101636171341, ..."
1,85,type of place of residence,type place residence,NaN,100.000,0.0,100.0,100.00,100.0,100.000,...,categorical,NaN,NaN,"{2.0: [('2.0: rural', 'rural'), ('2.0: other c...",15.0,"[rural, urban, other cities, other urban, yaou...","{'rural': 'rural', 'urban': 'urban', 'other ci...","{'..cotonou': 'cotonou', '..other urban': 'oth...","{'2.0: ..cotonou': 'cotonou', '3.0: ..other ur...","[-0.008523366414010525, 0.0022381108719855547,..."
2,61,household number,number,NaN,100.000,100.0,0.0,100.00,100.0,100.000,...,numeric,77502.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,"[-0.007650680374354124, -0.01233101636171341, ..."
3,9,cluster number,cluster number,NaN,100.000,100.0,0.0,100.00,100.0,100.000,...,numeric,61508057.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"[-0.011617615818977356, -0.010739745572209358,..."
4,102,result of household interview,result interview,NaN,100.000,0.0,100.0,100.00,100.0,100.000,...,numeric,NaN,NaN,NaN,2.0,[completed],{'completed': 'completed'},NaN,NaN,"[-0.01973319612443447, -0.027247991412878036, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,92,reason for preference of conical-shaped net: o...,other,NaN,0.201,0.0,0.2,0.56,100.0,0.685,...,numeric,NaN,NaN,NaN,2.0,"[no, yes]","{'no': 'no', 'yes': 'yes'}",NaN,NaN,"[-0.013715357519686222, -0.014628798700869083,..."
845,130,reason for preference of conical-shaped net: e...,easier to store when not hung,NaN,0.201,0.0,0.2,0.56,100.0,0.685,...,numeric,NaN,NaN,NaN,2.0,"[no, yes]","{'no': 'no', 'yes': 'yes'}",NaN,NaN,"[0.0030202805064618587, 0.001451018382795155, ..."
846,91,torch,torch,NaN,0.201,0.0,0.2,0.56,100.0,0.685,...,numeric,NaN,NaN,NaN,2.0,"[yes, no]","{'yes': 'yes', 'no': 'no'}",NaN,NaN,"[-0.00782063789665699, -0.03684137016534805, -..."
847,130,reason for preference of rectangular-shaped ne...,easier to hang,NaN,0.201,0.0,0.2,0.56,100.0,0.685,...,numeric,NaN,NaN,NaN,2.0,"[no, yes]","{'no': 'no', 'yes': 'yes'}",NaN,NaN,"[-0.006558047141879797, 0.01243473868817091, 0..."


In [ ]:
overwrite_pqt = True
integration_summary_f = integration_summary_f[:-4] + '_to_review_manually.pkl'
if not os.path.exists(integration_summary_f) or overwrite_pqt:

#    %mprun -f process_group_wrapper result_df = process_group_wrapper(integration_summary_df, file_paths, dataset_type, folder_path, input_dir)

    integration_summary_df = process_group_wrapper(integration_summary_df, file_paths, dataset_type, folder_path, input_dir)
    sorted_cols = ['similar_columns nr', 'column_name', 'column_name_normalized', 'overall indicator short', 
                   'overall indicator', 'numeric values as keys decision',  'available_data [%]',  'numeric [%]', 'string [%]', 
                'surveys with data in %', 'surveys with data percentage data available', 'available_data V7+ [%]', 'original % overwriting (as % of col2)', 
                'original % different values (as % of col2)', 
                'cosine string similarity answers', 'original different value counts', 
                'answer mapping for integration', 'answer mapping for integration not found', 'unique_answers wo keys', 'unique_answers',
                'integrated % overwriting (as % of col2)', 'integrated % different values (as % of col2)', 
                'integrated different value counts', 'replace_d numeric values as keys', 'numeric values as keys not found',
                'numeric similarity', 'mean', 'std', 'datatype similarity', 'cosine string similarity indicator'] 
    for c in sorted_cols.copy():
        if c not in integration_summary_df.columns:
            sorted_cols.remove(c)
            ic(c)
    ic(sorted_cols)
    cols = sorted_cols + [c for c in integration_summary_df.columns if c not in sorted_cols]
    integration_summary_df = integration_summary_df[cols]
    integration_summary_df_csv = integration_summary_df.drop(columns=['ada_embedding'])
    integration_summary_df_csv.to_csv(integration_summary_f[:-4] + '.csv', index=False)
    integration_summary_df.to_pickle(integration_summary_f)
else:
    integration_summary_df = pd.read_pickle(integration_summary_f)

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 1/1 [02:02<00:00, 122.81s/it]
0it [00:00, ?it/s]


In [ ]:
integration_summary_df.loc[:,'unique_answers wo keys list']

18                                                   NaN
532                                                  NaN
17                 [no, yes, yes public, yes other only]
71     [in yard plot, in dwelling, elsewhere, dans la...
72                                          [don't know]
                             ...                        
661                                            [no, yes]
790                                         [don't know]
825                                            [no, yes]
841                [private toilettes, public toilettes]
842                           [yes exclusive, no shared]
Name: unique_answers wo keys list, Length: 110, dtype: object